In [109]:
import torch
import torch.autograd as autograd
import torch.nn as nn
from torch import Tensor
import torch.optim as optim


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from sklearn.model_selection import train_test_split

import numpy as np
import time
import scipy.io

torch.set_default_dtype(torch.float)

# Random number generators in other libraries
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name()) 

cpu


In [266]:
# Lucas/this_script.py
import sys
import os
current_dir = os.getcwd()
print(current_dir)
# Add the 'functions' folder to the Python path
functions_dir = os.path.join(current_dir,'..', 'functions')
sys.path.append(functions_dir)
functions_dir = os.path.join(current_dir,'..', 'Mads')
sys.path.append(functions_dir)

# Now you can import the function
from Load_data import custom_csv_parser2, custom_csv_parser
from Load_data import data_split

pde_keys = ['D1', 'D2', 'I_sc','I_p', 'I_eff', 'G', 'G_sc']
patient_keys = ['tau1', 'tau2', 'Ci', 'p2', 'GEZI', 'EGP0', 'Vg', 'taum', 'tausc']
patient_keys_si = ['tau1', 'tau2', 'Ci', 'p2', 'Si', 'GEZI', 'EGP0', 'Vg', 'taum', 'tausc']

def dict_to_vec(X, t):
    """Converts a dictionary of time series data to a vector"""
    t = t.cpu().numpy().astype(int) if t.numel() > 1 else [int(t.item())]  # Ensure t is a list of integers
    X_vec = torch.zeros((len(t), len(pde_keys)), device=device)  # Initialize X_vec with the correct shape
    
    for i, ti in enumerate(t):
        for j, key in enumerate(pde_keys):
            value = torch.tensor(X[key][ti], device=device)  # Convert to torch.Tensor
            X_vec[i, j] = value
    
    return X_vec.to(device)

# Use the function
data = custom_csv_parser2('../Patient2.csv')
parms = custom_csv_parser('../Patient.csv')
Si_true = parms['Si']

X_train, X_test, ts_train, ts_test, ts = data_split(data)
G_true = X_train["G"]


c:\DeepLearning-PINNs\Lucas
Training data loaded successfully!


# Hyperparamters

In [267]:
Si0 = 0.1

p = 0.4
lr = 0.001

In [268]:
class DNN(nn.Module):
    '''Network'''
    def __init__(self, layers):
        super(DNN, self).__init__()

        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1] ) for i in range(len(layers)-1)])

        self.activation = nn.Tanh()

        self.dropout = nn.Dropout(p=p)

        'Initialization'
        for i in range(len(layers) - 1):
            nn.init.xavier_normal_(self.linears[i].weight, gain=1.0)
            nn.init.zeros_(self.linears[i].bias)

    def forward(self, x):
        for i in range(0, len(self.linears) - 1):
            x = self.linears[i](x)
            x = self.activation(x)
            x = self.dropout(x)
        x = self.linears[-1](x)
        return x

In [269]:
def pdes(u, nn_dt, Si):
    pdes = torch.zeros_like(u, device=device)
    D1 = u[:, 0]  # Example, assuming D1 is the first column of u
    D2 = u[:, 1]  # Example, assuming D2 is the second column of u
    I_sc = u[:, 2]  # Example, assuming I_sc is the third column of u
    I_p = u[:, 3]  # Example, assuming I_p is the fourth column of u
    I_eff = u[:, 4]  # Example, assuming I_eff is the fifth column of u
    G = u[:, 5]  # Example, assuming G is the sixth column of u
    G_sc = u[:, 6]  # Example, assuming G_sc is the seventh column of u
    
    # Convert parameters to tensors
    taum = torch.tensor(parms['taum'], device=device)
    tau1 = torch.tensor(parms['tau1'], device=device)
    tau2 = torch.tensor(parms['tau2'], device=device)
    p2 = torch.tensor(parms['p2'], device=device)
    Ci = torch.tensor(parms['Ci'], device=device)
    GEZI = torch.tensor(parms['GEZI'], device=device)
    EGP0 = torch.tensor(parms['EGP0'], device=device)
    Vg = torch.tensor(parms['Vg'], device=device)
    tausc = torch.tensor(parms['tausc'], device=device)

    d = 0
    meal = 0
    
    # Example assignment for all rows of pdes
    pdes[:, 0] = nn_dt[:, 0] + D1 / taum - d
    pdes[:, 1] = nn_dt[:, 1] - (D1 - D2) / taum
    pdes[:, 2] = nn_dt[:, 2] + I_sc / tau1 - meal / (tau1 * Ci)
    pdes[:, 3] = nn_dt[:, 3] - (I_sc - I_p) / tau2
    pdes[:, 4] = nn_dt[:, 4] + p2 * I_eff - p2 * Si * I_p
    pdes[:, 5] = nn_dt[:, 5] + (GEZI + I_eff) * G - EGP0 - 1000 * D2 / (Vg * taum)
    pdes[:, 6] = nn_dt[:, 6] - (G - G_sc) / tausc
    
    return pdes

class PINN():
    def __init__(self, layers):
        self.iter = 0

        self.Si = torch.tensor([Si0], requires_grad=True, device=device)
        self.Si = nn.Parameter(self.Si)
        self.nn = DNN(layers).to(device)
        self.nn.register_parameter('Si', self.Si)

    def data_loss(self, ts, data):
        u = self.nn(ts)  # Outputs ts x 7 array
        loss = 0
        for i,key in enumerate(pde_keys):
            loss += torch.mean((u[i] - data[key][int(ts[i])])**2)
        return loss

    def pde_loss(self, ts):
        
        ts = ts.clone().detach()
        ts = ts.requires_grad_(True)
        u = self.nn(ts)        
        nn_dt = torch.zeros_like(u, device=device)

        for i in range(u.shape[1]):
            nn_dt[:, i] = autograd.grad(u[:, i], ts, grad_outputs=torch.ones_like(u[:, i]), create_graph=True)[0].squeeze()

        pd = pdes(u, nn_dt, self.Si)
        loss = nn.MSELoss()(pd, torch.zeros_like(pd))
        return loss
    
    def loss(self, ts, data):
        return self.data_loss(ts, data) + self.pde_loss(ts)
    
    def test(self, t):
        #t = t.unsqueeze(0)
        X_pred = self.nn.forward(t)
        X_true = dict_to_vec(data, t)
        print(X_pred.shape)
        print(X_true.shape)
        error_vec = torch.linalg.norm((X_true-X_pred),2)/torch.linalg.norm(X_true,2)
        X_pred = X_pred.cpu().detach().numpy()
        return X_pred, error_vec


In [270]:
NN = PINN([1, 20, 20, 20, 20, 20, 20, 7])
params = list(NN.nn.parameters())
optimizer = optim.Adam(NN.nn.parameters(), lr=0.001)

'''Test Network works'''
pred, err = NN.test(torch.tensor([1], device=device, dtype=torch.float32))
print(f"""Prediction: {pred} \n Error: {err}""")
print("Current guess for Si: ", NN.Si.item())


torch.Size([7])
torch.Size([1, 7])
Prediction: [-1.6499155  -1.5664932   0.11730391 -0.42615038  1.2028909  -0.30301777
  1.4235082 ] 
 Error: 0.9957432746887207
Current guess for Si:  0.10000000149011612


In [274]:
start_time = time.time()
ts_train = torch.tensor(ts_train, device=device, dtype=torch.float32).reshape(-1,1)
ts_test = torch.tensor(ts_test, device=device, dtype=torch.float32)
epochs = 5000

for epoch in range(epochs):
    optimizer.zero_grad()
    loss = NN.loss(ts_train, parms)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}, Si: {NN.Si.item()}")

print("True Si: ", Si_true)

C:\Users\lucas\AppData\Local\Temp\ipykernel_14612\3976177905.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ts_train = torch.tensor(ts_train, device=device, dtype=torch.float32).reshape(-1,1)
C:\Users\lucas\AppData\Local\Temp\ipykernel_14612\3976177905.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ts_test = torch.tensor(ts_test, device=device, dtype=torch.float32)


Epoch: 0, Loss: 18787.552734375, Si: 0.013522883877158165
Epoch: 100, Loss: 19385.0546875, Si: 0.013351238332688808
Epoch: 200, Loss: 22027.24609375, Si: 0.012042965739965439
Epoch: 300, Loss: 20480.251953125, Si: 0.011882125400006771
Epoch: 400, Loss: 20833.556640625, Si: 0.011268348433077335
Epoch: 500, Loss: 19247.8828125, Si: 0.010700435377657413
Epoch: 600, Loss: 19115.572265625, Si: 0.01038329117000103
Epoch: 700, Loss: 16994.21875, Si: 0.01041802205145359
Epoch: 800, Loss: 15498.8837890625, Si: 0.009785013273358345
Epoch: 900, Loss: 19046.39453125, Si: 0.00953638181090355
Epoch: 1000, Loss: 16133.6064453125, Si: 0.00956802163273096
Epoch: 1100, Loss: 15366.216796875, Si: 0.008953992277383804
Epoch: 1200, Loss: 16231.6484375, Si: 0.008937248960137367
Epoch: 1300, Loss: 19329.80078125, Si: 0.008471050299704075
Epoch: 1400, Loss: 17158.162109375, Si: 0.008559755980968475
Epoch: 1500, Loss: 17218.439453125, Si: 0.008775832131505013
Epoch: 1600, Loss: 16591.84765625, Si: 0.0081925997